look here for all options:
http://open-platform.theguardian.com/documentation/search

In [1]:
import requests,json,pandas as pd,pickle

In [4]:
countries=pd.read_csv('../aux/countries.txt2',header=None)
countries=[m.lower() for m in countries[0]]
apiKeyInp=open('../aux/apiKey','r')

In [5]:
apiKey=apiKeyInp.readline().strip()
baseURL='http://content.guardianapis.com/search?api-key='+apiKey+'&show-blocks=all&page-size=50'

In [4]:
def get_req(Q,from_date,to_date):
    url=baseURL+'&q='+Q+'&from-date='+from_date+'&to-date='+to_date
    r=requests.get(url)
    rj=r.json()['response']
    return(rj)

def get_data(country,year):
    year=str(year)
    dd=['01-01','04-1','07-01','10-01','12-30']
    quarters=[(year+'-'+dd[i],year+'-'+dd[i+1]) for i in range(len(dd)-1)]
    out=[get_req(country,m[0],m[1]) for m in quarters]
    return(out)

In [5]:
def get_mention_graph(Q,year):
    dic={}
    r1=get_data(Q,year)
    pages=[rj['pages'] for rj in r1]
    res=[rj['results'] for rj in r1]
    cont=[[m['blocks']['body'] for m in resi] for resi in res]
    contText=[[m[0]['bodyTextSummary'] for m in conti if len(m)!=0] for conti in cont]
    cont_text=[]
    for j in range(4):
        m=''
        for i in contText[j]:
            m+=i
        cont_text+=[m]
    dic['pages']=pages
    dic['content']=cont_text
    cnt_counts=[[(m,ss.lower().count(m)) for m in countries] for ss in dic['content']]
    dic['cgraph']=[[(Q,m[0],m[1]) for m in ss if m[1]!=0] for ss in cnt_counts]
    return(dic)

In [6]:
out_dic={}
for Q in countries:
    for Y in range(2006,2017):
        kk=Q+"_"+str(Y)
        w=get_mention_graph(Q,Y)
        out_dic[kk]=w
        
pickle.dump(out_dic,open('graphs_2006_2016.pkl','wb'))